## 结构化数组

假设我们要保存这样的数据：

||name|age|wgt|
|---|---|---|---|
|0|dan|1|23.1|
|1|ann|0|25.1|
|2|sam|2|8.3|

希望定义一个一维数组，每个元素有三个属性name,age,wgt,此时我们需要使用结构化数组。

In [1]:
import numpy as np

定义数组a：

|0|1|2|3|
|---|---|---|---|
|1.0|2.0|3.0|4.0|

In [2]:
a  = np.array([1.0, 2.0, 3.0, 4.0], np.float32)

使用view方法，将a对应的内存按照复数来解释：

In [4]:
a.view(np.complex64)

array([ 1.+2.j,  3.+4.j], dtype=complex64)


|0|1|2|3|
|---|---|---|---|
|1.0|2.0|3.0|4.0|
|real|imag|real|imag|

事实上，我们可以把复数看成一个结构体，第一部分是实部，第二部分是虚部，这样这个数组便可以看成是一个结构化的数组。

换句话说，我们只需要换种方式解释这段内存，便可以得到结构化的数组的效果！

|0|1|2|3|
|---|---|---|---|
|1.0|2.0|3.0|4.0|
|mass|vol|mass|vol|

例如，我们可以将第一个浮点数解释为质量，第二个浮点数解释为速度，则这段内存还可以看成是包含两个域（质量和速度）的结构体。

In [5]:
my_dtype = np.dtype([('mass','float32'),('vol','float32')])

In [6]:
a.view(my_dtype)

array([(1.0, 2.0), (3.0, 4.0)], 
      dtype=[('mass', '<f4'), ('vol', '<f4')])

这里，我们使用dtype创造了自定义的结构类型，然后用自定义的结构来解释数组a所占的内存。

这里f4表示四个字节浮点数，<表示小字节序。

利用这个自定义的结构类型，我们可以这样初始化结构数组：

In [7]:
my_data = np.array([(1,1), (2,1), (2,1), (1,3)],my_dtype)

print(my_data)

[(1.0, 1.0) (2.0, 1.0) (2.0, 1.0) (1.0, 3.0)]


第一个元素：

In [8]:
my_data[0]

(1.0, 1.0)

得到第一个元素的速度信息，可以使用域的名称来索引：

In [9]:
my_data[0]['vol']

1.0

得到所有的质量信息：

In [11]:
my_data['mass']

array([ 1.,  2.,  2.,  1.], dtype=float32)

自定义排序规则，先按速度，再按质量：

In [13]:
my_data.sort(order=('vol','mass'))

print(my_data)

[(1.0, 1.0) (2.0, 1.0) (2.0, 1.0) (1.0, 3.0)]


回到最初的例子，定义一个人的结构类型：

In [14]:
person_dtype = np.dtype([('name','S10'),('age','int'),('weight','float')])

查看类型所占字节数：

In [15]:
person_dtype.itemsize

26

产生一个3 x 4 共12人的空结构体数组：

In [16]:
people = np.empty((3,4), person_dtype)

分别赋值：

In [17]:
people['name'] = [['Brad', 'Jane', 'John', 'Fred'],
                  ['Henry', 'George', 'Brain', 'Amy'],
                  ['Ron', 'Susan', 'Jennife', 'Jill']]

In [18]:
people['age'] = [[33, 25, 47, 54],
                 [29, 61, 32, 27],
                 [19, 33, 18, 54]]

In [19]:
people['weight'] = [[135., 105., 255., 140.],
                    [154., 202., 137., 187.],
                    [188., 135., 88., 145.]]

In [20]:
print(people)

[[(b'Brad', 33, 135.0) (b'Jane', 25, 105.0) (b'John', 47, 255.0)
  (b'Fred', 54, 140.0)]
 [(b'Henry', 29, 154.0) (b'George', 61, 202.0) (b'Brain', 32, 137.0)
  (b'Amy', 27, 187.0)]
 [(b'Ron', 19, 188.0) (b'Susan', 33, 135.0) (b'Jennife', 18, 88.0)
  (b'Jill', 54, 145.0)]]


In [21]:
people[-1,-1]

(b'Jill', 54, 145.0)

### 从文本中读取结构化数组

我们有这样一个文本：

In [22]:
%%writefile people.txt
name age weight
amy 11 38.2
jhon 10 40.3
bill 12 21.2

Writing people.txt


利用loadtxt指定数据类型，从这个文件中读取结构化数组：

In [23]:
person_dtype = np.dtype([('name','S10'),('age','int'),('weight','float')])

people = np.loadtxt('people.txt',
                    skiprows=1,
                    dtype=person_dtype)
people

array([(b'amy', 11, 38.2), (b'jhon', 10, 40.3), (b'bill', 12, 21.2)], 
      dtype=[('name', 'S10'), ('age', '<i8'), ('weight', '<f8')])

查看name域：

In [24]:
people['name']

array([b'amy', b'jhon', b'bill'], 
      dtype='|S10')

删除文件：

In [25]:
import os 
os.remove('people.txt')

对于下面的文件：

In [26]:
%%writefile wood.csv
item, material,number
100, oak, 38
110, maple, 14
120, oak, 7
145, birch, 3

Writing wood.csv


定义转换函数处理材料属性，使之对应一个整数：

In [51]:
tree_to_int = dict(oak = 1,
                   maple=2,
                   birch=3)
        
def convert(s):
    return tree_to_int.get(s,0)


使用genfromtxt载入数据，可以自动从第一行读入属性名称：

In [53]:
data = np.genfromtxt('wood.csv',
                     delimiter=',', #逗号分隔
                     dtype = np.int, #数据类型
                     names = True,  #从第一行读入属性名
                     converters={1:convert} #转换函数               
                    )
data

array([(100, 0, 38), (110, 0, 14), (120, 0, 7), (145, 0, 3)], 
      dtype=[('item', '<i8'), ('material', '<i8'), ('number', '<i8')])

 **出现异常:**虽然没有报错，但是输出的结果不对，转换函数转换出来的都是默认值。

* 测试转换函数：

In [55]:
convert('oak')

1

* 读取文件查看文件内容：

In [48]:
with open('wood.csv') as f:
    for line in f:
        print(line)

item, material,number

100, oak, 38

110, maple, 14

120, oak, 7

145, birch, 3


未发现问题，修改函数进行实验。将数据类型修改为None，转换函数修改到为{2:convert},输出结果如下：

In [57]:
data = np.genfromtxt('wood.csv',
                     delimiter=',', #逗号分隔
                     dtype = None, #数据类型
                     names = True,  #从第一行读入属性名
                     converters={2:convert} #转换函数               
                    )
data

array([(100, b' oak', 0), (110, b' maple', 0), (120, b' oak', 0),
       (145, b' birch', 0)], 
      dtype=[('item', '<i8'), ('material', 'S6'), ('number', '<i8')])

从上面的输出结果来看，每个名字中前面有一个空格，如b' maple'可能原因是因为这个空格，重新写入下文件试试：

In [58]:
%%writefile wood.csv
item, material,number
100,oak, 38
110,maple, 14
120,oak, 7
145,birch, 3

Overwriting wood.csv


In [66]:
data = np.genfromtxt('wood.csv',
                     delimiter=',', #逗号分隔
                     dtype = None, #数据类型
                     names = True,  #从第一行读入属性名
                     #converters={1:convert} #转换函数               
                    )
data

array([(100, b'oak', 38), (110, b'maple', 14), (120, b'oak', 7),
       (145, b'birch', 3)], 
      dtype=[('item', '<i8'), ('material', 'S5'), ('number', '<i8')])

In [69]:
print(data[1]['material'])
convert(data[1]['material'])

b'maple'


0

还是不行，读取原始数据进行了验证，结果也是默认值，所以问题已经非常明显了，应该是编码方式的不同导致了该问题。

下面就尝试重新定义函数，在函数内部进行编码转换：

In [70]:
tree_to_int = dict(oak = 1,
                   maple=2,
                   birch=3)
        
def convert(s):
    su = s.decode()  #默认转换为utf-8，可以不填，百度知道的
    return tree_to_int.get(su,0)

In [73]:
data = np.genfromtxt('wood.csv',
                     delimiter=',', #逗号分隔
                     dtype = np.int, #数据类型
                     names = True,  #从第一行读入属性名
                     converters={1:convert} #转换函数               
                    )
data

array([(100, 1, 38), (110, 2, 14), (120, 1, 7), (145, 3, 3)], 
      dtype=[('item', '<i8'), ('material', '<i8'), ('number', '<i8')])

通过测试，问题解决。

**总结：**
参考的教程是python2的教程，两者在某些方面还是有区别的，本次遇到的问题是因为str格式和bin格式在python3中做了区分。

关于格式转换，参考了[转换格式](http://www.cnblogs.com/txw1958/archive/2012/08/31/python3-bytes-string.html)的指导。

解决之后，了解了[python3中str和bin的区别](http://www.cnblogs.com/feixuelove1009/p/5521417.html)

也许也可以在genfromtxt中指定数据类型，自动将数据类型转换为str类型。暂时先不研究了。

str和bytes区别：

在python中是两种数据类型，str的编码为Unicode（万国码），而bytes可以定义为指定为utf-8或者gbk。使用writefile时默认保存的是utf-8编码数据类型。

* 1.在将字符串存入磁盘和从磁盘读取字符串的过程中，Python自动地帮你完成了编码和解码的工作，你不需要关心它的过程。

* 2.使用bytes类型，实质上是告诉Python，不需要它帮你自动地完成编码和解码的工作，而是用户自己手动进行，并指定编码格式。

* 3.Python已经严格区分了bytes和str两种数据类型，你不能在需要bytes类型参数的时候使用str参数，反之亦然。这点在读写磁盘文件时容易碰到。

查看域：

In [74]:
data['material']

array([1, 2, 1, 3])

删除文件：

In [75]:
import os
os.remove('wood.csv')

### 嵌套类型

有时候，结构数组中的域可能包括嵌套结构，例如，我们希望在二维平面上记录一个质点的位置和质量：

<table >
    <tr>
        <td>
            <table style="margin:0;border:0">
                <tr>
                    position
                </tr>
                <tr>
                    <td>x</td>
                    <td>y</td>
                </tr>
            </table>
        </td>
        <td>mass</td>
    </tr>
</table>

(这个表格，做的挫了点，先讲究着看吧，捂脸)


那么它的类型可以这样定义：

In [76]:
particle_dtype = ([('position',[('x','float'),
                                ('y','float')]),
                   ('mass','float')
                  ])

假设数据文件如下：

In [77]:
%%writefile data.txt
2.0 3.0 42.0
2.1 4.3 32.5
1.2 4.6 32.3
4.5 -6.4 23.3

Writing data.txt


读取数据：

In [79]:
data = np.loadtxt('data.txt', dtype=particle_dtype)

In [80]:
data

array([((2.0, 3.0), 42.0), ((2.1, 4.3), 32.5), ((1.2, 4.6), 32.3),
       ((4.5, -6.4), 23.3)], 
      dtype=[('position', [('x', '<f8'), ('y', '<f8')]), ('mass', '<f8')])

查看x轴：

In [81]:
data['position']['x']

array([ 2. ,  2.1,  1.2,  4.5])

删除生成文件：

In [82]:
import os
os.remove('data.txt')